In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [13]:
df = pd.read_csv('mushrooms.csv')

In [35]:
df.head(10)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [17]:
le = LabelEncoder()
ds = df.apply(le.fit_transform)

In [19]:
ds = ds.values

In [30]:
print(ds.shape)
ds

(8124, 23)


array([[1, 5, 2, ..., 2, 3, 5],
       [0, 5, 2, ..., 3, 2, 1],
       [0, 0, 2, ..., 3, 2, 3],
       ...,
       [0, 2, 2, ..., 0, 1, 2],
       [1, 3, 3, ..., 7, 4, 2],
       [0, 5, 2, ..., 4, 1, 2]])

In [31]:
Y = ds [:,0]
X = ds[:,1:]

In [32]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2)

In [33]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(6499, 22) (6499,)
(1625, 22) (1625,)


In [34]:
np.unique(y_train)

array([0, 1])

## Classifier

In [36]:
def prior_prob(y_train,label):
    total_examples = y_train.shape[0]
    class_examples = np.sum(y_train == label)
    
    return (class_examples)/float(total_examples)

In [37]:
def cond_probablity(x_train,y_train,feature_col,feature_val,label):
    x_filtered = x_train[y_train == label]
    numerator = np.sum(x_filtered[:,feature_col] == feature_val)
    denominator = np.sum(y_train == label)
    
    return numerator/float(denominator)

In [39]:
def Predict(x_train,y_train,x_test):
    classes = np.unique(y_train)
    n_features = x_train.shape[1]
    post_probs = []
    
    #Compute posterior for each class
    for label in classes:
        likelihood = 1.0
        for f in range(n_features):
            cond = cond_probablity(x_train,y_train,f,x_test[f],label)
            likelihood *= cond
            
        prior = prior_prob(y_train,label)
        post = likelihood*prior
        post_probs.append(post)
    
    pred = np.argmax(post_probs)
    return pred

In [42]:
out = Predict(x_train,y_train,x_test[1])
print(out,y_test[1])

1 1


In [43]:
def score(x_train,y_train,x_test,y_test):
    pred = []
    
    for i in range(x_test.shape[0]):
        pred_label = Predict(x_train,y_train,x_test[i])
        pred.append(pred_label)
        
    pred = np.array(pred)
    
    acc = np.sum(pred == y_test)/y_test.shape[0]
    return acc

In [45]:
accuracy = score(x_train,y_train,x_test,y_test)

In [46]:
accuracy*100

99.44615384615385